# Packages 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *


/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [2]:
ranker_train_data_dir = '../data/rank_train_data'
rank_model_version = 'rank_lgbm_v3'

In [3]:
debug = False

if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model_version = 'w2v_v3'
nic_model_version = 'nic'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100



# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")


sub_file = f'../data/sub_files/{rank_model_version}.parque'
eval_sub_file = f'../data/sub_files/eval_{rank_model_version}.parque'

In [4]:
! mkdir {rank_model_dir}

mkdir: cannot create directory ‘../model_training/rank_lgbm_v3’: File exists


# Read data

In [5]:

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))


In [6]:
! ls {ranker_train_data_dir}

eval.parquet  test.parquet  train.parquet


In [7]:
train_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train.parquet'), n_rows=SAMPLE_NUM)
eval_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'), n_rows=SAMPLE_NUM)
test_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'), n_rows=SAMPLE_NUM)

# Ranker 

## Config 

In [8]:
estimator = 1000
early_stop_rounds = 10
eval_frac = 0.1
if debug:
    early_stop_rounds = 5
    
target = 'target'
feature_cols = [
    # 'whether_w2v'
    # , 'whether_nic'
    'next_item_weight'
    , 'locale'
    , 'w2v_weight'
    , 'last_item_similarity'
    # , 'prev_item_similarity'	
    , 'prev_length'
]

categorical_feature=['locale']

## Train model

In [9]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    n_estimators=estimator, 
    importance_type='gain',
    eval_at=[3]
)

In [10]:
train_session_id = train_candidates.select(pl.col('session_id')).unique().collect()

session_num = train_session_id.shape[0]

In [11]:
random_pl = train_session_id.with_columns(
        pl.Series(name='random', values=np.random.uniform(size=session_num))
)

In [12]:
train_candidates = train_candidates.join(random_pl.lazy(), how='left', on='session_id')

sampled_train_candidates = train_candidates.filter(pl.col('random')>eval_frac).collect()
sampled_eval_candidates = train_candidates.filter(pl.col('random')<=eval_frac).collect()

In [13]:
print('Train')
print(sampled_train_candidates.shape)
print(sampled_train_candidates.select('target').mean())
print()
print()
print('Eval')
print(sampled_eval_candidates.shape)
print(sampled_eval_candidates.select('target').mean())

Train
(33235567, 13)
shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.079086 │
└──────────┘


Eval
(3696135, 13)
shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.079044 │
└──────────┘


In [14]:
session_lengths_train = sampled_train_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()
session_lengths_eval = sampled_eval_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()

In [15]:
ranker = ranker.fit(
    X=sampled_train_candidates[feature_cols].to_pandas(),
    y=sampled_train_candidates[target].to_pandas(),
    group=session_lengths_train,
    eval_set=[(sampled_train_candidates[feature_cols].to_pandas(), sampled_train_candidates[target].to_pandas()),
           (sampled_eval_candidates[feature_cols].to_pandas(), sampled_eval_candidates[target].to_pandas())
             ],
    eval_group=[session_lengths_train,
                session_lengths_eval
               ]
    , categorical_feature=categorical_feature
    , callbacks=[early_stopping(early_stop_rounds)]
)

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:621: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['locale']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overridin

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's ndcg@3: 0.868848	valid_1's ndcg@3: 0.868639


In [16]:
print(sampled_train_candidates.estimated_size(unit='gb'))
del sampled_train_candidates

5.943528516218066


In [17]:
print(sampled_eval_candidates.estimated_size(unit='gb'))
del sampled_eval_candidates

0.6618443410843611


## Save model 

In [18]:
debug

False

In [19]:
if not debug:
    joblib.dump(
            value=ranker,
            filename=os.path.join(rank_model_dir, 'model.pkl')
    )

In [20]:
if not debug:
    del ranker

## Load Model 

In [21]:
if not debug:
    ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))
ranker

LGBMRanker(eval_at=[3], importance_type='gain', metric='ndcg',
           n_estimators=1000, objective='lambdarank')

### Importance 

In [22]:
impotant_df = pd.DataFrame(
    {
        'features': ranker.feature_name_,
        'importance': ranker.feature_importances_
    }
).sort_values('importance', ascending=False)
impotant_df

,features,importance
0,next_item_weight,1.691156e+07
3,last_item_similarity,1.652890e+05
2,w2v_weight,1.092929e+05
4,prev_length,5.259802e+03
1,locale,0.000000e+00


# Eval ranker 

In [23]:
def rank_rec(candidate_pl, feature_cols, ranker, topn=100):

    candidate_pl = candidate_pl.collect()
    print(f"candidate pl shape: {candidate_pl.shape}")
    inference = ranker.predict(candidate_pl[feature_cols].to_pandas())
    test_result = (candidate_pl
         .lazy()
         .with_columns(
             pl.Series(name='predict', values=inference)
         )
         .with_columns(
            pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
         )
         .sort(['session_id', 'rank'])
         .filter(pl.col('rank')<=topn)
         .groupby(['session_id'])
         .agg(
             pl.col('next_item_prediction')
             , pl.col('next_item').unique()#.arr.get(0).cast(pl.Utf8)
         ).with_columns(
             pl.col('next_item').arr.get(0).alias('next_item')
         )
    )
    return test_result


In [24]:
ranker_eval_pl = rank_rec(candidate_pl=eval_candidates,
                         feature_cols=feature_cols
                         , ranker=ranker
                        # , topn=10
                         )

candidate pl shape: (37901553, 11)


In [25]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
326443,0.31974,0.497404,0.527066


# Test inference 

In [26]:
test_candidates = test_candidates.collect()
test_candidates.shape

(38892094, 9)

In [27]:
inference = ranker.predict(test_candidates[feature_cols].to_pandas())

In [28]:
test_result = (test_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=100)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test_result.head().collect()

In [29]:
test_pl.schema

{'prev_items': Unknown, 'locale': Utf8, 'session_id': Int64}

In [30]:
predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [31]:
predictions.shape

(316971, 2)

In [32]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend
if not debug:
    predictions.to_parquet(sub_file, engine='pyarrow')

# Submit result 

In [34]:
# # You can submit with aicrowd-cli, or upload manually on the challenge page.
!aicrowd submission create -c task-1-next-product-recommendation -f {sub_file}

rank_lgbm_v3.parque ━━━━━━━━━━━━━━━ 100.0% • 300.9/300.8 MB • 2.9 MB/s • 0:00:0000:0100:07
                                                                                 ╭─────────────────────────╮                                                                                 
                                                                                 │ Successfully submitted! │                                                                                 
                                                                                 ╰─────────────────────────╯                                                                                 
                                                                                       Important links                                                                                       
┌──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────